In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 500)

from sklearn import svm
import re
from shutil import copyfile
import matplotlib.pyplot as plt
import pydicom
from tqdm import tqdm

## Notebook to trim the dataset from NDA to only the needed files

Most of this should be standard except at towards the bottom there are two ways to verify correct files. 

In [2]:
# for 2274:
path = '../physcosis/Package_1187332/'

# should be standard
test_file = 'panss01.txt'
subject_file = 'ndar_subject01.txt'
image_file = 'image03.txt'




In [3]:
# opensfiles using pandas
# drop rows that are fully duplicated from the start --> not need 
ndar_subject = pd.read_csv(os.path.join(path, subject_file), delimiter = '\t')
print(ndar_subject.shape)
ndar_subject = ndar_subject.drop_duplicates()
print(ndar_subject.shape)
test = pd.read_csv(os.path.join(path, test_file), delimiter = '\t')
print(test.shape)
test = test.drop_duplicates()
print(test.shape)
image03 = pd.read_csv(os.path.join(path, image_file), delimiter = '\t')
print(image03.shape)
image03 = image03.drop_duplicates()
print(image03.shape)

(4404, 26)
(4404, 26)
(1779, 42)
(1779, 42)
(9395, 51)
(9395, 51)


In [4]:
# shows the different types of images available for the data package
image03.pivot_table(columns = ['image_description'], aggfunc = 'size')

image_description
ADNI Double_TSE SENSE                                                      561
ADNI MPRAGE SENSE                                                           45
ADNI MPRAGE pg7 SENSE                                                      516
AX T2                                                                       16
Axial PD-T2 TSE                                                            118
Axial PD-T2 TSE no angle                                                    73
Axial PD-T2 TSE_ NO ANGULATION                                               4
Axial PD-T2 TSE_NO_ANGLE                                                    26
B1-Calibration Body                                                         98
B1-Calibration Body_COIL                                                    25
B1-Callibration Head                                                        98
B1-Callibration Head_COIL                                                   24
Bsnip Ax PD-T2 TSE NoAng          

In [5]:
image03.pivot_table(columns = ['scan_type'], aggfunc = 'size')

scan_type
MR diffusion          3882
MR structural (T1)    2902
MR structural (T2)     977
Type of Scan             1
fMRI                  1633
dtype: int64

In [6]:
image03 = image03[image03['scan_type'].isin(['MR structural (T1)'])]
image03.shape

(2902, 51)

In [7]:
data_dir = os.path.join(path, 'image03')

#### CHANGE BASED ON WHAT 'image_file' COLUMN LOOKS LIKE IN image03 (corrects the file path to be on local machine) ####

# for 2274:
regex ='s3://NDAR_.*/submission_.[0-9]*/'


rows_with_bad_path = []
for index, row in image03.iterrows():
    image_path = re.sub(regex, '', row['image_file'])
    full_path = os.path.join(data_dir, image_path)
#     print(full_path)
    if not os.path.exists(full_path):
        print(image_path, index, row['image_file'])
        rows_with_bad_path.append(index)
rows_with_bad_path

[]

In [8]:
# drops rows based on rows with bad path and replaces old paths with local paths

image03 = image03.drop(rows_with_bad_path)
l = []

for index, row in image03.iterrows():
    image_path = re.sub(regex, '', row['image_file'])
    full_path = os.path.join(data_dir, image_path)
    l.append(full_path)
image03['image_file'] = l
image03.shape

(2902, 51)

In [9]:
# count number of unique subject ids in the panss test 
print(test['src_subject_id'].nunique())
print(test.columns)
print(test.shape)
# I think only these are needed change is using another test
test = test[['src_subject_id', 'interview_age', 'panss_general',
       'panss_negative', 'panss_positive', 'pos_p1', 'pos_p2', 'pos_p3',
       'pos_p4', 'pos_p5', 'pos_p6', 'pos_p7', 'neg_n1', 'neg_n2', 'neg_n3',
       'neg_n4', 'neg_n5', 'neg_n6', 'neg_n7', 'gps_g1', 'gps_g2', 'gps_g3',
       'gps_g4', 'gps_g5', 'gps_g6', 'gps_g7', 'gps_g8', 'gps_g9', 'gps_g10',
       'gps_g11', 'gps_g12', 'gps_g13', 'gps_g14', 'gps_g15', 'gps_g16']]
test.pivot_table(columns = ['src_subject_id', 'interview_age'], aggfunc = 'size')
# drop duplicate tests for subjects
test = test.drop_duplicates(subset = ['src_subject_id', 'interview_age'])
print(test.shape)

968
Index(['collection_id', 'panss01_id', 'dataset_id', 'subjectkey',
       'src_subject_id', 'interview_age', 'sex', 'panss_general',
       'panss_negative', 'panss_positive', 'panss_total', 'pos_p1', 'pos_p2',
       'pos_p3', 'pos_p4', 'pos_p5', 'pos_p6', 'pos_p7', 'neg_n1', 'neg_n2',
       'neg_n3', 'neg_n4', 'neg_n5', 'neg_n6', 'neg_n7', 'gps_g1', 'gps_g2',
       'gps_g3', 'gps_g4', 'gps_g5', 'gps_g6', 'gps_g7', 'gps_g8', 'gps_g9',
       'gps_g10', 'gps_g11', 'gps_g12', 'gps_g13', 'gps_g14', 'gps_g15',
       'gps_g16', 'collection_title'],
      dtype='object')
(1779, 42)
(968, 35)


In [10]:
print(ndar_subject.shape)
ndar_subject.pivot_table(columns = ['src_subject_id', 'interview_age'], aggfunc = 'size')
print(ndar_subject.shape)
ndar_subject = ndar_subject.drop_duplicates(subset = ['src_subject_id', 'interview_age'])
print(ndar_subject.shape)

(4404, 26)
(4404, 26)
(2416, 26)


In [11]:
res1 = image03.merge(ndar_subject, on = ['src_subject_id', 'interview_age'], how = 'inner')
print(res1.shape)
res2 = res1.merge(test,  on = ['src_subject_id', 'interview_age'], how = 'inner')


# print(res2['src_subject_id'].nunique())
print(res2.columns)
print(res2.shape)

master_table = res2
# master_table.pivot_table(columns = ['src_subject_id', 'interview_age'], aggfunc = 'size')

(2822, 75)
Index(['collection_id_x', 'image03_id', 'dataset_id_x', 'subjectkey_x',
       'src_subject_id', 'interview_age', 'sex_x', 'comments_misc',
       'image_file', 'image_description',
       ...
       'gps_g7', 'gps_g8', 'gps_g9', 'gps_g10', 'gps_g11', 'gps_g12',
       'gps_g13', 'gps_g14', 'gps_g15', 'gps_g16'],
      dtype='object', length=108)
(1341, 108)


In [12]:
master_table.pivot_table(columns = ['image_description'], aggfunc = 'size')

image_description
ADNI Double_TSE SENSE         291
ADNI MPRAGE SENSE              36
ADNI MPRAGE pg7 SENSE         255
B1-Calibration Body            52
B1-Calibration Body_COIL       13
B1-Callibration Head           51
B1-Callibration Head_COIL      13
Bsnip Sag MPRAGE-like ADNI    113
MPRAGE                         85
MPRAGE Repeat                  13
MPRAGE_SAG                      2
MPRAGE_SAG Repeat               2
MPRAGE_repeat                  50
SAG B1 CAL 8CH                  2
SAG B1 CAL BODY                 2
SAG MPRAGE                     22
Sagittal MPRAGE                 1
Sagittal MPRAGE-like ADNI      17
T1 MPRAGE                     168
T1 MPRAGE Repeat                2
WIP ADNI Double_TSE SENSE      36
WIP ADNI MPRAGE pg7 SENSE      36
loc                             5
localizer                      72
sagB1calbody                    1
sagb1cal 8ch                    1
dtype: int64

## Verify Image types

If image types are not known, ex. you do not know if an image description in the 'image_description' column is an actual T1 MRI scan, then run the following cells. This will copy one example from each type from the 'image_description' column and move it to a TEST folder and unzip and attempt to convert to niftii. TODO: What do do for non dicomm datasets. 



If you already are sure of the image types that are needed from the pivot table above, then skip the following 9 cells and enter the good_formats variable in manually. 

In [ ]:
#### Optionally check if all the remaining types can be changed to niftii format properly and filter


test_dir = os.path.join(data_dir, 'TEST')

if not os.path.exists(test_dir):
    os.mkdir(test_dir)
test_dir




In [ ]:
single_example = image03.drop_duplicates(subset = ['image_description'], keep='first')
single_example.shape

In [ ]:
for i,row in single_example.iterrows():
    file_name = row['image_file'].split('/')[-1]
    
    folder = os.path.join(test_dir, file_name.split('.')[0])
    if not os.path.exists(folder):
        os.system('mkdir ' + folder)
        os.system('unzip ' + row['image_file'] + ' -d ' + folder)
        print('Unziped and moved ', file_name)

In [ ]:
new_path = []
for i,row in single_example.iterrows():
    folder = row['image_file'].split('/')[-1].split('.')[0]
    p = os.path.join(test_dir, folder)
    nifti = os.path.join(p, '{}.nii'.format(folder))
    if not os.path.exists(nifti):
        print('Converting ', folder, ' from dicom to niftii.')
        try:
            dicom2nifti.dicom_series_to_nifti(p, nifti, reorient_nifti=True)
            new_path.append(nifti)
        except: 
            new_path.append('FAILED')
            print(folder, ' Failed.')
        
    else: 
        new_path.append(nifti)

In [ ]:
single_example['newpath'] = new_path

In [ ]:
single_example = single_example[single_example.newpath != 'FAILED']

In [ ]:
good_formats = list(single_example['image_description'])

# manually remove any that we don't need 
good_formats.remove('ep2d_diff_convert_Allegra_B-SNIP')
good_formats.remove('ep2d_diff_convert_Allegra_B-SNIP_ADC')
good_formats.remove('ep2d_diff_convert_Allegra_B-SNIP_TRACEW')
good_formats.remove('ep2d_diff_convert_Allegra_B-SNIP_FA')


good_formats

In [ ]:
master_table = master_table[master_table['image_description'].isin(good_formats)]

In [ ]:
master_table = master_table.drop_duplicates(subset = ['src_subject_id', 'interview_age'])
master_table.shape

### If manually choosing good_format to keep in master table, change the list below: 

In [13]:
###### manually choose which formats: #####


good_formats = ['T1 MPRAGE',
 'T1 MPRAGE Repeat',
 'MPRAGE Repeat',
 'MPRAGE_SAG Repeat',
 'Bsnip Sag MPRAGE-like ADNI',
 'Sagittal MPRAGE-like ADNI',
 'Sagittal MPRAGE']

In [14]:
master_table = master_table[master_table['image_description'].isin(good_formats)]

In [15]:
master_table = master_table.drop_duplicates(subset = ['src_subject_id', 'interview_age'])

In [16]:
master_table.shape

(301, 108)

### Move the remaining images to a clean folder and save master_table as a csv

In [26]:
clean_dir = os.path.join(data_dir, 'CLEAN')

if not os.path.exists(clean_dir):
    os.mkdir(clean_dir)


new_paths = []
for i,row in master_table.iterrows():
    
    file_name = row['image_file'].split('/')[-1]
    os.system('cp -r ' + row['image_file'] + ' ' + os.path.join(clean_dir, file_name))
#     print('cp -r ' + row['image_file'] + ' ' + os.path.join(clean_dir, file_name))
    new_paths.append(file_name)

In [29]:
master_table['image_file'] = new_paths

In [33]:
master_table.to_csv(os.path.join(clean_dir, 'master_table.csv'), sep = '\t', index = False)